<a href="https://colab.research.google.com/github/seungwoosoon/SmartFarmProjectAi/blob/main/Yolo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## YOLOn8 커스텀 모델링

간단한 설명

In [ ]:
"""
# YOLO_detecting_disease&undernourishment
Author: 유지찬, 오연희
Date: 2025-08-07
Purpose: 토마토 병해 및 영양결핍 탐지 모델 훈련

## Requirements
- ultralytics: 8.3.174
- torch: 2.6.0+cu124
- torchvision: 0.21.0+cu124
- numpy: 2.0.2
- opencv-python: 4.12.0.88
- pandas: 2.2.2

## Dataset
- Google Drive: /MyDrive/detecting_disease_undernourishment(YOLO)/dataset
- Classes: 6개
- names: ['정상', '토마토잿빛곰팡이병', '토마토흰가루병', '다량원소결핍(N)', '다량원소결핍(P)', '다량원소결핍(K)']]
- Train images: 5000
- Val images: 848

## Model
- Base Model: YOLOv8s
- Input Size: 640x640
- Batch Size: 16
- Epochs: 50
"""

라이브러리 설치

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

구글 드라이브 마운트

데이터셋을 구글드라이브에서 코랩 로컬 디스크로 복사

data.yaml 작성

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("구글 드라이브 마운트 완료")

# 데이터셋을 코랩 로컬 디스크로 복사
SOURCE_PATH = "/content/drive/My Drive/detecting_disease&undernourishment(YOLO)/dataset_split"
DEST_PATH = "/content/dataset_temp"

# 기존 데이터 삭제 및 디렉토리 생성
!rm -rf "$DEST_PATH"
!mkdir -p "$DEST_PATH"

!echo "=== 데이터 복사 중입니다. 약 20GB로 10분 정도 소요될 수 있습니다. ==="
!cp -r "$SOURCE_PATH/part_01" "$DEST_PATH/"
!cp -r "$SOURCE_PATH/part_02" "$DEST_PATH/"
!cp -r "$SOURCE_PATH/part_14" "$DEST_PATH/"
!echo "데이터 복사 완료."

# data.yaml 파일 생성 (오류 방지)
yaml_content = """
train:
 - /content/dataset_temp/part_01
 - /content/dataset_temp/part_02
val:
 - /content/dataset_temp/part_14

nc: 6
names: ['정상', '토마토잿빛곰팡이병', '토마토흰가루병', '다량원소결핍(N)', '다량원소결핍(P)', '다량원소결핍(K)']
"""

with open('/content/data.yaml', 'w') as f:
    f.write(yaml_content)

print("data.yaml 파일 생성 완료.")

Mounted at /content/drive
구글 드라이브 마운트 완료
=== 데이터 복사 중입니다. 약 10GB로 10분 정도 소요될 수 있습니다. ===
^C
^C
데이터 복사 완료.
data.yaml 파일 생성 완료.


데이터 훈련(구글 드라이브에 결과저장)

In [ ]:
# ultralytics 라이브러리 설치
!pip install ultralytics

# 학습 시작
from ultralytics import YOLO
import os

# 모델 로드 (yolov8s.pt는 자동으로 다운로드됩니다)
model = YOLO('yolov8s.pt')

# 모델 훈련 시작
model.train(
    data='/content/data.yaml',  # 로컬 디스크의 data.yaml 파일을 사용
    epochs=50,
    imgsz=640,
    batch=16,
    project='/content/drive/MyDrive/tomato_disease_detection',
    name='yolov8s_test_run'
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
구글 드라이브 마운트 성공!


Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_test_run3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, 

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               


  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768,

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.4±0.1 ms, read: 173.8±187.6 MB/s, size: 3386.7 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/50      4.79G     0.7506     0.6007      1.163         23        640: 100%|██████████| 313/313 [52:23<00:00, 10.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [04:01<00:00, 17.25s/it]

                   all        424        424          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/50      4.82G     0.7365     0.5685      1.151         33        640:  83%|████████▎ | 259/313 [45:28<07:43,  8.58s/it]

학습 재개 코드(중단된 경우)

- 학습이 중간에 끊겼을 때, 위에 저장한 last.pt파일을 불러와서 이어서 진행하는 코드

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import os

# 구글 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

# 저장된 last.pt 파일의 경로 설정
checkpoint_path = '/content/drive/MyDrive/tomato_disease_detection/yolov8s_test_run/weights/last.pt'

# 체크포인트가 존재하면 로드, 없으면 yolov8s.pt 로드
if os.path.exists(checkpoint_path):
    print(f"'{checkpoint_path}'에서 모델을 로드하여 학습을 이어서 진행합니다.")
    model = YOLO(checkpoint_path)
else:
    print("저장된 모델을 찾을 수 없습니다. yolov8s.pt 모델을 새로 로드합니다.")
    model = YOLO('yolov8s.pt')

# 훈련 시작 (epochs 값은 50으로 유지하면 됨)
model.train(
    data='/content/data.yaml',
    epochs=50,
    imgsz=640,
    project='/content/drive/MyDrive/tomato_disease_detection',
    name='yolov8s_test_run'
)